In [5]:
year = 2023
day = 14

In [68]:
from aocd import submit
from aocd.models import Puzzle
import numpy as np

puzzle = Puzzle(year=year, day=day)
data = puzzle.input_data
# data = puzzle.examples[0].input_data

data = data.strip()
data = data.split("\n")
# data = data.split("\n\n")
# data = [np.array([list(line) for line in d.split("\n")]) for d in data]
data = [list(line) for line in data]
data = np.array(data)
data

array([['.', '.', '#', ..., '#', '.', '.'],
       ['.', 'O', '.', ..., '.', '.', '.'],
       ['.', '.', '.', ..., 'O', '.', '.'],
       ...,
       ['O', '.', '.', ..., '.', 'O', 'O'],
       ['.', '#', '.', ..., '.', 'O', '.'],
       ['.', 'O', '.', ..., '.', 'O', '.']], dtype='<U1')

In [69]:

def tilt(data, direction):
    H, W = data.shape

    if direction == "N":
        offset = (-1, 0)
        coord_order = [(y, x) for y in range(1, H) for x in range(W)]
    elif direction == "S":
        offset = (1, 0)
        coord_order = [(y, x) for y in range(H-2, -1, -1) for x in range(W)]
    elif direction == "E":
        offset = (0, 1)
        coord_order = [(y, x) for x in range(W-2, -1, -1) for y in range(H)]
    elif direction == "W":
        offset = (0, -1)
        coord_order = [(y, x) for x in range(1, W) for y in range(H)]
    else:
        raise ValueError("Invalid direction")

    moving = True
    while moving:
        moving = False
        for y, x in coord_order:
                if data[y, x] == "O" and data[y+offset[0], x+offset[1]] == ".":
                    moving = True
                    data[y+offset[0], x+offset[1]] = "O"
                    data[y, x] = "."
    return data

data

array([['.', '.', '#', ..., '#', '.', '.'],
       ['.', 'O', '.', ..., '.', '.', '.'],
       ['.', '.', '.', ..., 'O', '.', '.'],
       ...,
       ['O', '.', '.', ..., '.', 'O', 'O'],
       ['.', '#', '.', ..., '.', 'O', '.'],
       ['.', 'O', '.', ..., '.', 'O', '.']], dtype='<U1')

In [70]:
def calculate_score(data):
    H, W = data.shape
    total = 0
    for x in range(W):
        for y in range(H):
            if data[y, x] == "O":
                total += H-y
    return total

In [71]:
answer = calculate_score(tilt(data.copy(), "N"))
submit(answer, part="a", year=year, day=day)

aocd will not submit that answer again. At 2023-12-14 00:12:34.190351-05:00 you've previously submitted 113424 and the server responded with:
That's the right answer!  You are one gold star closer to restoring snow operations. [Continue to Part Two]


In [57]:
def cycle(data):
    data = tilt(data, "N")
    data = tilt(data, "W")
    data = tilt(data, "S")
    data = tilt(data, "E")
    return data

In [58]:
cycle(data.copy())

array([['.', '.', '.', '.', '.', '#', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '#', '.', '.', '.', 'O', '#'],
       ['.', '.', '.', 'O', 'O', '#', '#', '.', '.', '.'],
       ['.', 'O', 'O', '#', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', 'O', 'O', 'O', '#', '.'],
       ['.', 'O', '#', '.', '.', '.', 'O', '#', '.', '#'],
       ['.', '.', '.', '.', 'O', '#', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', 'O', 'O', 'O', 'O'],
       ['#', '.', '.', '.', 'O', '#', '#', '#', '.', '.'],
       ['#', '.', '.', 'O', 'O', '#', '.', '.', '.', '.']], dtype='<U1')

In [78]:
def cycle_until_stable(data):
    scores = []
    for i in range(150):
        s = calculate_score(data)
        print(i, i%9, s)
        data = cycle(data)
        if s in scores:
            return scores
        scores.append(s)
    return scores


In [79]:
stable = cycle_until_stable(data.copy())

0 0 105314
1 1 104345
2 2 104167
3 3 103875
4 4 103585
5 5 103442
6 6 103254
7 7 103071
8 8 102834
9 0 102569
10 1 102275
11 2 102022
12 3 101737
13 4 101517
14 5 101336
15 6 101159
16 7 100999
17 8 100857
18 0 100681
19 1 100541
20 2 100341
21 3 100177
22 4 100002
23 5 99835
24 6 99657
25 7 99495
26 8 99299
27 0 99138
28 1 98971
29 2 98821
30 3 98651
31 4 98491
32 5 98291
33 6 98128
34 7 97958
35 8 97819
36 0 97662
37 1 97540
38 2 97391
39 3 97267
40 4 97129
41 5 97037
42 6 96934
43 7 96858
44 8 96773
45 0 96736
46 1 96663
47 2 96626
48 3 96573
49 4 96580
50 5 96583
51 6 96622
52 7 96647
53 8 96681
54 0 96704
55 1 96760
56 2 96807
57 3 96864
58 4 96888
59 5 96899
60 6 96898
61 7 96859
62 8 96813
63 0 96776
64 1 96765
65 2 96770
66 3 96849
67 4 96963
68 5 97051
69 6 97150
70 7 97218
71 8 97277
72 0 97277


In [76]:
for i, s in enumerate(stable):
    print(i%9, s)

0 105314
1 104345
2 104167
3 103875
4 103585
5 103442
6 103254
7 103071
8 102834
0 102569
1 102275
2 102022
3 101737
4 101517
5 101336
6 101159
7 100999
8 100857
0 100681
1 100541
2 100341
3 100177
4 100002
5 99835
6 99657
7 99495
8 99299
0 99138
1 98971
2 98821
3 98651
4 98491
5 98291
6 98128
7 97958
8 97819
0 97662
1 97540
2 97391
3 97267
4 97129
5 97037
6 96934
7 96858
8 96773
0 96736
1 96663
2 96626
3 96573
4 96580
5 96583
6 96622
7 96647
8 96681
0 96704
1 96760
2 96807
3 96864
4 96888
5 96899
6 96898
7 96859
8 96813
0 96776
1 96765
2 96770
3 96849
4 96963
5 97051
6 97150
7 97218
8 97277
0 97277
1 97257
2 97202
3 97170
4 97161
5 97153
6 97149
7 97156
8 97137
0 97121
1 97072
2 97042
3 96986
4 96921
5 96852
6 96780
7 96714
8 96643
0 96566
1 96500
2 96432
3 96359
4 96276
5 96215
6 96174
7 96121
8 96079
0 96044
1 96014
2 95990
3 95972
4 95962
5 95961
6 95981
7 96001
8 96020
0 96014
1 96003
2 95985
3 95971
4 95962
5 95961
6 95981
7 96001
8 96020
0 96014
1 96003
2 95985
3 95971
4 95962
5

In [77]:
1_000_000_000 % 9
answer = 96003

1